In [1]:
import pandas as pd
import numpy as np
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import vaex
vaex.multithreading.thread_count_default = 8
import vaex.ml
import pickle

In [2]:
pristine = pd.read_parquet('/arc/projects/k-pop/pristine/Pristine_CFIS.parquet',
                           columns=["CaHK_0","pmra_1","pmdec_1","g_chp_psf","r_chp_psf","i_chp_psf","z_chp_psf","y_chp_psf","MAG_AUTO",
                                    "phot_g_mean_mag_1","phot_bp_mean_mag_1","phot_rp_mean_mag_1"])

In [3]:
pristine = pristine.loc[(pristine["g_chp_psf"] < 1e19) & (pristine["r_chp_psf"] < 1e19) & (pristine["i_chp_psf"] < 1e19) & \
                        (pristine["z_chp_psf"] < 1e19) & (pristine["y_chp_psf"] < 1e19) & (pristine["MAG_AUTO"] < 95) & \
                        (np.isnan(pristine["phot_g_mean_mag_1"]) == False) & (np.isnan(pristine["phot_bp_mean_mag_1"]) == False) & \
                        (np.isnan(pristine["pmra_1"]) == False) & (np.isnan(pristine["pmdec_1"]) == False) & \
                        (np.isnan(pristine["phot_rp_mean_mag_1"]) == False)]

In [4]:
df = vaex.from_pandas(pd.DataFrame({'PMRA': pristine['pmra_1'],
                   'PMDec': pristine['pmdec_1'],
                   'G_gaia': pristine['CaHK_0']-pristine['phot_g_mean_mag_1'],
                   'B_gaia': pristine['CaHK_0']-pristine['phot_bp_mean_mag_1'],
                   'R_gaia': pristine['CaHK_0']-pristine['phot_rp_mean_mag_1'],
                   'G_pan': pristine['CaHK_0']-pristine['g_chp_psf'],
                   'R_pan': pristine['CaHK_0']-pristine['r_chp_psf'],
                   'I_pan': pristine['CaHK_0']-pristine['i_chp_psf'],
                   'Z_pan': pristine['CaHK_0']-pristine['z_chp_psf'],
                   'Y_pan': pristine['CaHK_0']-pristine['y_chp_psf'],
                   'AUTO_cfis': pristine['CaHK_0']-pristine['MAG_AUTO']}))

In [5]:
print(df.shape)
data = df.dropna()
print(data.shape)

(3718245, 11)
(3718245, 11)


In [6]:
scaled_star_data = StandardScaler().fit_transform(data)

In [7]:
%%time
filename = 'unsupervisedpickle.obj'
with open(filename,'rb') as file_object:
    raw_data = file_object.read()

reducer = pickle.loads(raw_data)

<timed exec>:5: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.


Sun Apr  3 01:02:19 2022 Building and compiling search function


AttributeError: 'NNDescent' object has no attribute 'parallel_batch_queries'

In [8]:
embedding = reducer.transform(scaled_star_data)

NameError: name 'reducer' is not defined

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(embedding[:, 0], embedding[:, 1], cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar()
plt.savefig('/arc/home/aydanmckay/bigdatapristinecolours.png')

In [ ]:
%%time
# sdss = pd.read_csv('/arc/home/aydanmckay/leftjoinpristinespecobj.csv',
#                    usecols=["CaHK_0_p","pmra_1_p","pmdec_1_p","g_chp_psf_p","r_chp_psf_p","i_chp_psf_p","z_chp_psf_p","y_chp_psf_p","MAG_AUTO_p",
#                             "phot_g_mean_mag_1_p","phot_bp_mean_mag_1_p","phot_rp_mean_mag_1_p","CLASS_s"])
sspp = pd.read_csv('/arc/home/aydanmckay/leftjoinpristinesspp.csv',
                   usecols=["CaHK_0_p","pmra_1_p","pmdec_1_p","g_chp_psf_p","r_chp_psf_p","i_chp_psf_p","z_chp_psf_p","y_chp_psf_p","MAG_AUTO_p",
                            "phot_g_mean_mag_1_p","phot_bp_mean_mag_1_p","phot_rp_mean_mag_1_p","AFE_s","FEH_ADOP_s","TEFF_ADOP_s","LOGG_ADOP_s"])

In [ ]:
# sdss = sdss.loc[(sdss["g_chp_psf_p"] < 1e19) & (sdss["r_chp_psf_p"] < 1e19) & (sdss["i_chp_psf_p"] < 1e19) & \
#                         (sdss["z_chp_psf_p"] < 1e19) & (sdss["y_chp_psf_p"] < 1e19) & (sdss["MAG_AUTO_p"] < 95) & \
#                         (np.isnan(sdss["phot_g_mean_mag_1_p"]) == False) & (np.isnan(sdss["phot_bp_mean_mag_1_p"]) == False) & \
#                         (np.isnan(sdss["pmra_1_p"]) == False) & (np.isnan(sdss["pmdec_1_p"]) == False) & \
#                         (np.isnan(sdss["phot_rp_mean_mag_1_p"]) == False)]

In [ ]:
sspp = sspp.loc[(sspp["g_chp_psf_p"] < 1e19) & (sspp["r_chp_psf_p"] < 1e19) & (sspp["i_chp_psf_p"] < 1e19) & \
                        (sspp["z_chp_psf_p"] < 1e19) & (sspp["y_chp_psf_p"] < 1e19) & (sspp["MAG_AUTO_p"] < 95) & \
                        (np.isnan(sspp["phot_g_mean_mag_1_p"]) == False) & (np.isnan(sspp["phot_bp_mean_mag_1_p"]) == False) & \
                        (np.isnan(sspp["pmra_1_p"]) == False) & (np.isnan(sspp["pmdec_1_p"]) == False) & \
                        (np.isnan(sspp["phot_rp_mean_mag_1_p"]) == False)]

In [ ]:
# sdss = sdss.replace({'CLASS_s': np.nan}, 'Unknown')
# sdss.head()

In [ ]:
sspp = sspp.replace({'AFE_s': np.nan}, -90)
sspp = sspp.replace({'FEH_ADOP_s': np.nan}, -90)
# sspp.head()

In [ ]:
# df_joined = vaex.from_pandas(pd.DataFrame({'PMRA': sdss['pmra_1_p'],
#                    'PMDec': sdss['pmdec_1_p'],
#                    'G_gaia': sdss["CaHK_0_p"]-sdss['phot_g_mean_mag_1_p'],
#                    'B_gaia': sdss["CaHK_0_p"]-sdss['phot_bp_mean_mag_1_p'],
#                    'R_gaia': sdss["CaHK_0_p"]-sdss['phot_rp_mean_mag_1_p'],
#                    'G_pan': sdss["CaHK_0_p"]-sdss['g_chp_psf_p'],
#                    'R_pan': sdss["CaHK_0_p"]-sdss['r_chp_psf_p'],
#                    'I_pan': sdss["CaHK_0_p"]-sdss['i_chp_psf_p'],
#                    'Z_pan': sdss["CaHK_0_p"]-sdss['z_chp_psf_p'],
#                    'Y_pan': sdss["CaHK_0_p"]-sdss['y_chp_psf_p'],
#                    'AUTO_cfis': sdss["CaHK_0_p"]-sdss['MAG_AUTO_p']},
#                    index=np.arange(0,len(sdss))))
# df2 = pd.DataFrame({'Class': sdss['CLASS_s']}, index = np.arange(0,len(sdss)))

In [ ]:
df_joined_fe = vaex.from_pandas(pd.DataFrame({'PMRA': sspp['pmra_1_p'],
                      'PMDec': sspp['pmdec_1_p'],
                      'G_gaia': sspp["CaHK_0_p"]-sspp['phot_g_mean_mag_1_p'],
                      'B_gaia': sspp["CaHK_0_p"]-sspp['phot_bp_mean_mag_1_p'],
                      'R_gaia': sspp["CaHK_0_p"]-sspp['phot_rp_mean_mag_1_p'],
                      'G_pan': sspp["CaHK_0_p"]-sspp['g_chp_psf_p'],
                      'R_pan': sspp["CaHK_0_p"]-sspp['r_chp_psf_p'],
                      'I_pan': sspp["CaHK_0_p"]-sspp['i_chp_psf_p'],
                      'Z_pan': sspp["CaHK_0_p"]-sspp['z_chp_psf_p'],
                      'Y_pan': sspp["CaHK_0_p"]-sspp['y_chp_psf_p'],
                      'AUTO_cfis': sspp["CaHK_0_p"]-sspp['MAG_AUTO_p']},
                      index=np.arange(0,len(sspp))))
df2_afe = pd.DataFrame({'a/Fe': sspp['AFE_s']}, index = np.arange(0,len(sspp)))
df2_feh = pd.DataFrame({'Fe/H': sspp['FEH_ADOP_s']}, index = np.arange(0,len(sspp)))
df2_logg = pd.DataFrame({'logg': sspp['LOGG_ADOP_s']}, index = np.arange(0,len(sspp)))
df2_teff = pd.DataFrame({'Teff': sspp['TEFF_ADOP_s']}, index = np.arange(0,len(sspp)))

In [ ]:
# print(df_joined.shape)
# df_joined = df_joined.dropna()
# print(df_joined.shape)
# print(df2.shape)
# df2 = df2.dropna()
# print(df2.shape)

In [ ]:
print(df_joined_fe.shape)
df_joined_fe = df_joined_fe.dropna()
print(df_joined_fe.shape)
print(df2_afe.shape)
df2_afe = df2_afe.dropna()
print(df2_afe.shape)
print(df2_feh.shape)
df2_feh = df2_feh.dropna()
print(df2_feh.shape)
print(df2_logg.shape)
df2_logg = df2_logg.dropna()
print(df2_logg.shape)
print(df2_teff.shape)
df2_teff = df2_teff.dropna()
print(df2_teff.shape)

In [ ]:
# scaled_star_data_joined = StandardScaler().fit_transform(df_joined)
# embedding_joined = reducer.transform(scaled_star_data_joined)

In [ ]:
%%time
scaled_star_data_joined_fe = StandardScaler().fit_transform(df_joined_fe)
embedding_joined_fe = reducer.transform(scaled_star_data_joined_fe)

In [ ]:
# df2cmap = df2.replace('Unknown',0)
# df2cmap = df2cmap.replace('STAR',1)
# df2cmap = df2cmap.replace('GALAXY',2)
# df2cmap = df2cmap.replace('QSO',3)
# df2cmap.value_counts()

In [ ]:
# fig, ax = plt.subplots(figsize=(12,12));
# cax = ax.scatter(embedding_joined[:, 0], embedding_joined[:, 1], c=df2cmap['Class'], cmap='Spectral', s=5);
# ax.set_aspect('equal', 'datalim');
# cbar = fig.colorbar(cax, boundaries=np.arange(5)-0.5, ticks=[0, 1, 2, 3]);
# cbar.ax.set_yticklabels(['Unknown','Star','Galaxy','QSO']);
# ax.set_title("SpecObjs Catalogue Left Joined onto Pristine Dataset, Coloured by SpecObjs Class")
# # plt.savefig('/arc/home/aydanmckay/pmspecobjsmappedtopristinecolours.png')

In [ ]:
# mask = df2cmap['Class'] != 0

In [ ]:
# fig, ax = plt.subplots(figsize=(12,12));
# cax = ax.scatter(embedding_joined[:, 0][mask], embedding_joined[:, 1][mask], c=df2cmap['Class'][mask], cmap='Spectral', s=5);
# ax.set_aspect('equal', 'datalim');
# cbar = fig.colorbar(cax, boundaries=np.arange(1,5)-0.5, ticks=[1, 2, 3]);
# cbar.ax.set_yticklabels(['Star','Galaxy','QSO']);
# ax.set_title("SpecObjs Catalogue Left Joined onto Pristine Dataset, Coloured by SpecObjs Class");
# plt.savefig('/arc/home/aydanmckay/pmspecobjsmappedtopristinecoloursnounknowns.png')

In [ ]:
# mask_afe = df2_afe['a/Fe'] > -9

In [ ]:
# plt.figure(figsize=(12,12))
# plt.scatter(embedding_joined_fe[:, 0][mask_afe], embedding_joined_fe[:, 1][mask_afe],c=df2_afe['a/Fe'][mask_afe], cmap='Spectral', s=5)
# plt.axvline(-1)
# plt.axhline(-1)
# plt.gca().set_aspect('equal', 'datalim')
# plt.colorbar()
# plt.grid()
# plt.title(r"SpecObjs Catalogue Left Joined onto Pristine Dataset, Coloured by SSPP $\alpha$/Fe")
# plt.savefig('/arc/home/aydanmckay/pmssppmappedtopristinecoloursafe.png')

In [ ]:
mask_feh = df2_feh['Fe/H'] > -9

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(embedding[:, 0], embedding[:, 1], c='k', alpha=0.1, s=5)
plt.scatter(embedding_joined_fe[:, 0][mask_feh], embedding_joined_fe[:, 1][mask_feh],c=df2_feh['Fe/H'][mask_feh], cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar()
plt.title("SDSS DR17 Catalogue Left Joined onto Pristine Dataset, Coloured by SEGUE Fe/H")
plt.savefig('/arc/home/aydanmckay/pmssppmappedtopristinecoloursoverplot.png')

In [ ]:
sm_ll_feh = []
sm_ll_logg = []
sm_ll_teff = []
for x,y,feh,logg,teff in zip(embedding_joined_fe[:, 0][mask_feh],embedding_joined_fe[:, 1][mask_feh],
                             df2_feh['Fe/H'][mask_feh],df2_logg['logg'][mask_feh],df2_teff['Teff'][mask_feh]):
    if x < -1 and y < -1:
        sm_ll_feh.append(feh)
        sm_ll_logg.append(logg)
        sm_ll_teff.append(teff)

In [ ]:
plt.scatter(sm_ll_teff,sm_ll_logg)
plt.title('Effective Temperature vs. logg on the Small Low Metallicity Blob')
plt.xlabel('Effective temperature')
plt.ylabel('logg')
plt.grid()
plt.savefig('teffloggblob.png')

In [ ]:
plt.scatter(sm_ll_logg,sm_ll_feh)
plt.title('logg vs. Metallicity on the Small Low Metallicity Blob')
plt.ylabel('Metallicity')
plt.xlabel('logg')
plt.grid()
plt.savefig('loggfehblob.png')

In [ ]:
# len(df2_logg['logg'][mask_feh])

In [ ]:
# len(df2_feh['Fe/H'][mask_feh])

In [ ]:
# len(embedding_joined_fe[:, 0][mask_feh])